# advanced CNN

# AlexNet
 - first use Relu
 - norm layers
 - drop out
 - SGD
 - 7 cnn -> ensemble

# VGGNet
  - filter를 더 작게 하여 여러번 해준다.
  - ex) 7 * 7을 한번하는 것 보다 3 * 3을 여러번 한다.
     - 같은 receptive filter를 추출해도 param의 개수가 적다


# googlenet

 - inception module 을 사용
   - 한 layer에 동시에 여러 filter를 사용
   - 그후 모든 결과를 concatenate한다.
   - Bottle neck layer를 이용하여 input의 dense를 줄어 연산량을 줄인다.
      - 인위적으로 1 * 1 conv fiter 32개를 씌어 dense를 줄임

 - no fc layers : computational efficieny를 높임 마지막에서만 사용 


 # ResNet
  - very deep network using residual connections
  - 깊은 뉴런이 overfitting 문제는 있어도 train에 대해서는 더욱 잘 맞추어야 하지만, 그렇지 못하는 문제가 발생했다.
  - otimization의 문제로 확인됨.
  - deep network -> problem -> residual connections
  - residual connections
    - input x의 값을 output f(x)에 값해 더해준다.
    - 즉, 현재의 정보도 기억하면서 학습을 진행하면, layer가 깊어져도 학습이 잘진해이 될것이라고 판단한다.
    - googlenet과 동일하게 bottle nect을 이용하여 연산량을 줄여준다.

In [6]:
# ResNet 

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy 

In [2]:
def conv3x3(in_planes,out_planes, stride =1):
    return nn.Conv2d(inplanes, out_plances, kernel_size = 3, stride = stride,
                   padding = 1, bias = False)
  
def conv1x1(in_planes,out_planes, stride =1):
    return nn.Conv2d(inplanes, out_plances, kernel_size = 1, stride = stride,
                   padding = 1, bias = False)

In [3]:
# 50개 이상의 layer를 가진 ResNet architecture에서 computational efficiency를 증가시키기 위해,
# 3x3 convolution layer 앞뒤로 1x1 convolution layer를 추가한 Bottleneck module을 구현
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride = 1, downsample = None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample # 차원이 바뀔때만 사용 마지막 input x+ out일 때
        self.conv2 = conv3x3(planes, planes)
        self.stride = stride

    def forward(self,x):
        identity = x

        out = self.conv1(x)
        out = self.bn(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn(out)
    
        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return out



In [4]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [5]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x